In [9]:
# Secure way to import key

from secretapikey import openai_api_key
import os

os.environ["OPENAI_API_KEY"] = openai_api_key

In [10]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

# LLM'S

In [4]:
llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Maharaja's Palace"


# PROMPTS

In [62]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a unique and contemporary name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a unique and contemporary name for this.


In [63]:
llm("I want to open a restaurant for Indian food. Suggest a fancy name for this.")

'\n\n"Spice Fusion Bistro"'

In [11]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)

chain.run("Mexican")

/Users/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'\n\n"El Jardín de Sabores" (The Garden of Flavors)'

In [30]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """ Recommend 15 menu items for {restaurant_name}. Just the names as a list"""
    )

food_items_chain = LLMChain(llm=llm, prompt = prompt_template_items)

In [14]:
chain = SimpleSequentialChain(chains = [name_chain,food_items_chain])
response = chain.run("Indian")
print(response)



1. Chicken Tikka Masala
2. Lamb Vindaloo
3. Vegetable Korma
4. Tandoori Shrimp
5. Paneer Tikka
6. Samosas
7. Palak Paneer
8. Aloo Gobi
9. Butter Chicken
10. Biryani
11. Naan Bread
12. Mango Lassi
13. Chana Masala
14. Malai Kofta
15. Gulab Jamun


In [19]:
chain = SimpleSequentialChain(chains = [name_chain,food_items_chain])
response = chain.run("Mexican")
print(response)

.

1. Tacos al Pastor
2. Enchiladas Verdes
3. Carne Asada
4. Chile Relleno
5. Camarones al Mojo de Ajo
6. Quesadillas de Huitlacoche
7. Sopa de Tortilla
8. Ceviche de Pescado
9. Fajitas de Pollo
10. Tamales de Mole
11. Burrito de Carnitas
12. Pozole
13. Tostadas de Tinga
14. Arroz con Pollo
15. Flan de Caramel


In [35]:
# Initialize the LLM with desired parameters
llm = OpenAI(temperature=0.7)

# Define the prompt template for generating restaurant names
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Recommend a fancy name for this."
)

# Define the name chain
name_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_name,
    output_key='restaurant_name'
)

# Define the prompt template for generating menu items
prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Recommend 15 menu items for {restaurant_name}"
)

# Define the food items chain
food_items_chain = LLMChain(
    llm=llm,
    prompt=prompt_template_items,
    output_key='menu_items'
)

# SEQUENTIAL CHAIN

In [36]:
# Define the SequentialChain with correct input and output handling
sequential_chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)

# Provide the initial input and run the chain
result = sequential_chain({'cuisine': 'Arabic'})

# Output the result
restaurant_name = result['restaurant_name'].strip()
menu_items = result['menu_items']

# Split the menu items into a list
menu_items_list = menu_items.split('\n') if '\n' in menu_items else menu_items.split(',')

# Remove empty items and strip extra whitespace
menu_items_list = [item.strip() for item in menu_items_list if item.strip()]

# Format and print the menu items with numbers
formatted_menu_items = "\n".join(f"{i+1}. {item}" for i, item in enumerate(menu_items_list))

print(f"Restaurant Name: {restaurant_name}\n")
print("Menu Items:\n")
print(menu_items_list)


Restaurant Name: "Sahara Spice"

Menu Items:

['1. Moroccan Spiced Lamb Tagine', '2. Tandoori Chicken Skewers', '3. Egyptian Falafel Wrap', '4. Spicy Tunisian Shakshuka', '5. Algerian Couscous with Vegetables', '6. Moroccan Harissa Chicken Sandwich', "7. Lebanese Za'atar Roasted Vegetables", '8. Tunisian Spiced Lamb Kebabs', '9. Egyptian Koshari Bowl', '10. North African Spiced Lentil Soup', '11. Moroccan Lamb and Apricot Tajine', '12. Spicy Chicken Shawarma Wrap', '13. Tunisian Spicy Grilled Shrimp', '14. Moroccan Vegetable Couscous Salad', '15. Harissa Hummus and Pita Platter']


# AGENTS

In [60]:
from langchain.agents import AgentType, initialize_agent, create_react_agent, load_tools
from langchain.llms import OpenAI

In [69]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"],llm=llm)


agent = initialize_agent(tools, 
                 llm,
                 agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True
                         
                )

agent.run("When is Elon Musk birthday and What is his age in 2024?")



> Entering new AgentExecutor chain...
 We can use Wikipedia to find the information about Elon Musk's birthday and his current age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp., formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world; as of June 2024, Forbes estimates his net worth to be US$210 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received 

"Elon Musk's age in 2024 will be 53 years old."

In [71]:
from secretserpapikey import serpapi_key

os.environ['SERPAPI_API_KEY'] = serpapi_key

In [80]:
# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"],llm=llm)


agent = initialize_agent(tools, 
                 llm,
                 agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True
                         
                )

agent.run("What was the GDP of UK 2023 and add 5 to it")



> Entering new AgentExecutor chain...
 I need to find the GDP of UK for 2023 first
Action: Search
Action Input: "GDP UK 2023"
Observation: GDP of the UK 1948-2023 The gross domestic product of the United Kingdom in 2023 was 2.274 trillion British pounds, a slight increase when compared to the previous year, when the size of the UK economy was 2.27 trillion pounds.
Thought: Now I need to add 5 to the GDP
Action: Calculator
Action Input: 2.274 trillion + 5
Observation: Answer: 2274000000005.0
Thought: I now know the final answer
Final Answer: The GDP of UK 2023 plus 5 is 2.274 trillion British pounds.

> Finished chain.


'The GDP of UK 2023 plus 5 is 2.274 trillion British pounds.'

In [81]:
dir(chain)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [83]:
from langchain.memory import ConversationBufferMemory

In [87]:
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory = memory)

name = chain.run("Mexican")
print(name)



"El Sabor de México" (The Flavor of Mexico)


In [88]:
name = chain.run("American")
print(name)



"Stateside Eats"


In [90]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor de México" (The Flavor of Mexico)
Human: American
AI: 

"Stateside Eats"


# Restriction buffer size so that the number of token generated into memory is not huge and is cost effective as well

In [93]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = OpenAI(temperature = 0.7))

print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [107]:
convo.run("Who is the current chess grandmaster")


'   As I mentioned before, the current chess grandmaster is still Magnus Carlsen. Did you know that he has been the world champion since 2013 and has successfully defended his title in three consecutive matches? He is truly a dominant force in the world of chess.'

In [96]:
convo.run("Who defeated Magnus Carlsen recently?")

' As I mentioned before, Magnus Carlsen was recently defeated by Wesley So. Did you know that So is currently ranked #8 in the world and is the only player to have defeated Carlsen in a classical time control game in the past two years? It was an impressive win for So and he is definitely a rising star in the chess world.'

In [99]:
convo.run("Who won the national elections in India 2024 and who is the prime minister?")

' I am not able to accurately answer that question as the national elections in India in 2024 have not yet taken place. The current prime minister of India is Narendra Modi, but it is uncertain if he will still hold that position in 2024. '

In [100]:
convo.run("Which Indian chess player defeated the current grandmaster")

'  The current chess grandmaster is Magnus Carlsen, and he has not been defeated by an Indian player recently. However, Indian chess player Viswanathan Anand has defeated Carlsen in the past, becoming the undisputed world champion in 2008. Anand is a five-time world champion and is considered one of the greatest chess players in history.'

In [103]:
print(convo.memory.buffer)

Human: Who is the current chess grandmaster
AI:  The current chess grandmaster is Magnus Carlsen. He is a Norwegian chess player and has held the title since 2013. He has an Elo rating of 2882, making him the highest-rated player in history. He has won numerous prestigious tournaments and is known for his strategic and aggressive playing style.
Human: Who is the current chess grandmaster
AI:  As I mentioned earlier, the current chess grandmaster is Magnus Carlsen. Did you know that he became a grandmaster at the age of 13, making him the second youngest person to achieve this title? He also holds the record for the longest unbeaten streak in competitive chess, with 125 games without a loss.
Human: Who defeated Magnus Carlsen recently?
AI:  Magnus Carlsen was recently defeated by American chess player Wesley So in the finals of the Skilling Open tournament in November 2020. However, Carlsen still maintains his top ranking as the current chess grandmaster.
Human: Who defeated Magnus Carl

In [105]:
from langchain.memory import ConversationBufferWindowMemory

In [112]:
memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(llm = OpenAI(temperature = 0.7), memory = memory)

In [113]:
convo.run("Who is the current chess grandmaster")

' The current chess grandmaster is Magnus Carlsen from Norway. He has held the title since 2013 and has a current rating of 2872, making him the highest rated player in history. He is known for his strategic and aggressive playing style and has won numerous tournaments and titles throughout his career.'

In [114]:
convo.run("Who won the national elections in India 2024 and who is the prime minister?")

' In 2024, the national elections in India were won by the Bharatiya Janata Party (BJP) led by Prime Minister Narendra Modi. Modi has been the Prime Minister of India since 2014 and is known for his economic reforms and strong stance on national security. He won a landslide victory in the 2024 elections, securing his second term in office.'

In [115]:
convo.run("Which Indian chess player defeated the current grandmaster")

" I'm sorry, I do not have information on the current grandmaster of chess. However, some notable Indian chess players include Viswanathan Anand, Pentala Harikrishna, and Vidit Santosh Gujrathi. Anand is a former world champion and is considered one of the greatest chess players of all time. Harikrishna is currently the highest-ranked Indian chess player and Gujrathi is known for his aggressive playing style."

In [116]:
print(convo.memory.buffer)

Human: Which Indian chess player defeated the current grandmaster
AI:  I'm sorry, I do not have information on the current grandmaster of chess. However, some notable Indian chess players include Viswanathan Anand, Pentala Harikrishna, and Vidit Santosh Gujrathi. Anand is a former world champion and is considered one of the greatest chess players of all time. Harikrishna is currently the highest-ranked Indian chess player and Gujrathi is known for his aggressive playing style.
